In [1]:
from transformers import AutoTokenizer

In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
tokenizer("Hello World")

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [5]:
tokens = tokenizer.tokenize("hello world")
tokens

['hello', 'world']

In [6]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [7]:
ids

[7592, 2088]

In [8]:
tokenizer.convert_ids_to_tokens(ids)

['hello', 'world']

In [9]:
tokenizer.decode(ids)

'hello world'

In [10]:
ids = tokenizer.encode("hello world")
ids

[101, 7592, 2088, 102]

In [11]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'hello', 'world', '[SEP]']

In [12]:
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [13]:
model_inputs = tokenizer("hello world")
model_inputs

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [14]:
data = [
  "I like cats.",
  "Do you like cats too?",
]
tokenizer(data)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
from transformers import AutoModelForSequenceClassification

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
outputs = model(**model_inputs)

AttributeError: 'list' object has no attribute 'size'

In [18]:
model_inputs = tokenizer("hello world", return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [19]:
# the default was to create a binary classifier!
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1524,  0.4722]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.0461, 0.3467, 0.3426]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
outputs.logits

tensor([[0.0461, 0.3467, 0.3426]], grad_fn=<AddmmBackward0>)

In [23]:
outputs['logits']

tensor([[0.0461, 0.3467, 0.3426]], grad_fn=<AddmmBackward0>)

In [24]:
outputs[0]

tensor([[0.0461, 0.3467, 0.3426]], grad_fn=<AddmmBackward0>)

In [25]:
outputs.logits.detach().cpu().numpy()

array([[0.04609854, 0.34672004, 0.34257415]], dtype=float32)

In [26]:
data = [
  "I like cats.",
  "Do you like cats too?",
]
model_inputs = tokenizer(data, return_tensors='pt')
model_inputs

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [27]:
model_inputs = tokenizer(
    data, padding=True, truncation=True, return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [28]:
model_inputs['input_ids']

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])

In [29]:
model_inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [30]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.0642, 0.4265, 0.2897],
        [0.0753, 0.4055, 0.2678]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)